In [57]:
import string
import math
import numpy as np

In [58]:
docs = []
query = ""
stopwords = set(['is', 'was', 'the', 'are', 'a', 'an'])

In [59]:
for i in range(1, 5):
    f = open(f"C:\D drive/PSG/sem8/20XW86-IR-LAB/catest1/documents/d{i}.txt", 'r')
    docs.append(f.read())

In [60]:
f = open(f"C:\D drive/PSG/sem8/20XW86-IR-LAB/catest1/documents/query.txt", 'r')
query = f.read()

In [61]:
docs

['Reuters, dataset, Reuters, benchmark, dataset',
 'Reuters, dataset, class, Reuters, class',
 'document, _multiple, dataset, document',
 'Reuters document, document, class']

In [62]:
query

'Reuters, document, dataset'

In [63]:
processed_docs = []

In [64]:
def transform_text(text):
    text = text.split()
    y = []
    
    for word in text:
        char_list = []
        for c in word:
            if c not in string.punctuation:
                char_list.append(c)
        
        lcase_word = ''.join(char_list[:]).lower()
        
        if lcase_word.isalnum() and lcase_word not in stopwords:
            y.append(lcase_word)

    return ' '.join(y)

In [65]:
transform_text(docs[0])

'reuters dataset reuters benchmark dataset'

In [66]:
for doc in docs:
    processed_docs.append(transform_text(doc))

In [67]:
processed_docs

['reuters dataset reuters benchmark dataset',
 'reuters dataset class reuters class',
 'document multiple dataset document',
 'reuters document document class']

In [68]:
# TF-IDF score calc.
# Wij = (TFk/LENk)*log(N+1/DFk+0.5)
# TFi - term freq within doc.
# LENi - len of document.
# N - total docs
# DFk - document frequency of the word K.

In [69]:
def TF(word, wordcount_map):
    return wordcount_map[word]

def DF(word, corpus_docmaps):
    count = 0
    
    for hm in corpus_docmaps:
        if word in hm:
            count += 1
    
    return count

def calculate_weights(doc, wordcount_map, corpus_docmaps):
    vector = [0]*len(wordcount_map)
    
    mymap = {}
    
    for i in doc.split():
        if i in mymap:
            mymap[i] +=1
        else:
            mymap[i] = 1
    
    for word in doc.split():
        TFik = TF(word, wordcount_map)
        LENi = len(doc)
        N = len(corpus_docmaps)
        DFk = DF(word, corpus_docmaps)
        
        try:
            index = list(wordcount_map.keys()).index(word)
            vector[index] = (TFik/LENi)*(math.log(N+1)/(0.5+DFk))
        except ValueError:
            pass
    
    return vector

In [70]:
word_countmap = {}

for doc in processed_docs:
    for word in doc.split():
        if word in word_countmap:
            word_countmap[word] += 1
        else:
            word_countmap[word] = 1

corpus_docmaps = []

for doc in processed_docs:
    hm = {}
    
    for word in doc.split():
        if word in hm:
            hm[word] += 1
        else:
            hm[word] = 1
    
    corpus_docmaps.append(hm)


doc_matrix = []
for doc in processed_docs:
    vector = calculate_weights(doc, word_countmap, corpus_docmaps)
    doc_matrix.append(vector)
    
print(doc_matrix)

[[0.05607797604300001, 0.04486238083440001, 0.026169722153400007, 0, 0, 0], [0.06569134336465715, 0.05255307469172572, 0, 0.055180728426312005, 0, 0], [0, 0.05409875335912942, 0, 0, 0.07573825470278119, 0.03155760612615883], [0.07416764573429033, 0, 0, 0.062300822416803875, 0.08306776322240517, 0]]


In [72]:
query_vector = calculate_weights(transform_text(query), word_countmap, corpus_docmaps)

In [73]:
query_vector

[0.09579987574012501, 0.0766399005921, 0, 0, 0.10729586082894, 0]

In [74]:
def cosine_similarity(vec1, vec2):
    dot_prod = sum([x*y for x,y in zip(vec1, vec2)])
    mag1 = sum([x*x for x in vec1])**0.5
    mag2 = sum([y*y for y in vec2])**0.5
    
    return dot_prod/mag1*mag2

In [81]:
res = []

In [82]:
for i,dv in enumerate(doc_matrix):
    res.append([i+1,cosine_similarity(dv, query_vector)])

In [83]:
sorted(res, key= lambda x:x[1], reverse=True)

[[4, 0.020459465635116483],
 [3, 0.02035239940183222],
 [1, 0.018786943639801365],
 [2, 0.016719605955572377]]